In [45]:
# connect to test.db sqlite, create session, then run "get all users" with session.Query
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from models import Base, Cast, User, Reaction, Location, parent_association
import pandas as pd


engine = create_engine('sqlite:///new.db')
Session = sessionmaker(bind=engine)
session = Session()


In [7]:
"""
Place all models in the notebook so Copilot knows what it's working with

Here are all the available models:
parent_association = Table('parent_association', Base.metadata,
                           Column('parent_hash', String,
                                  ForeignKey('casts.hash')),
                           Column('cast_hash', String,
                                  ForeignKey('casts.hash'))
                           )


# figure out ancestor hashes later
class Cast(Base):
    __tablename__ = 'casts'
    hash = Column(String, primary_key=True)
    thread_hash = Column(String, ForeignKey(
        'casts.hash'))
    parent_hash = Column(String, ForeignKey(
        'casts.hash'), nullable=True)
    text = Column(String)
    timestamp = Column(BigInteger)
    author_fid = Column(BigInteger, ForeignKey(
        'users.fid'))
    author = relationship('User', back_populates='casts')
    reactions = relationship('Reaction', back_populates='target')
    children_hashes = relationship("Cast", secondary=parent_association,
                                   primaryjoin=(
                                       hash == parent_association.c.parent_hash),
                                   secondaryjoin=(
                                       hash == parent_association.c.cast_hash),
                                   backref="parent_casts")


class Reaction(Base):
    __tablename__ = 'reactions'
    hash = Column(String, primary_key=True)
    reaction_type = Column(String)  # like & recast
    timestamp = Column(BigInteger)
    target_hash = Column(String, ForeignKey('casts.hash'))
    author_fid = Column(BigInteger, ForeignKey('users.fid'))
    target = relationship('Cast', back_populates='reactions')


class Location(Base):
    __tablename__ = 'locations'
    place_id = Column(String, primary_key=True)
    description = Column(String)
    users = relationship('User', backref='location')


class User(Base):
    __tablename__ = 'users'
    fid = Column(BigInteger, primary_key=True)
    username = Column(String)
    display_name = Column(String)
    verified = Column(Boolean, default=False)
    pfp_url = Column(String, nullable=True)
    follower_count = Column(BigInteger)
    following_count = Column(BigInteger)
    bio_text = Column(String, nullable=True)
    location_place_id = Column(String, ForeignKey(
        'locations.place_id'), nullable=True)
    casts = relationship('Cast', back_populates='author')

"""

'\nHere are all the available models:\n'

In [14]:
# get all users where they have location with description that contains "USA"
# description_location is from the Location model
# location_place_id is from the User model

locations = session.query(Location).filter(
    Location.description.contains("USA")).all()

# print users with locations
users = session.query(User).filter(
    User.location_place_id.in_([location.place_id for location in locations])).all()

df = pd.DataFrame([user.__dict__ for user in users])
# print(df)

# make df with username, location column

new_df = pd.DataFrame(columns=["username", "location"])
for user in users:
    new_df = pd.concat([new_df, pd.DataFrame({"username": [user.username], "location": [user.location.description]})], ignore_index=True)

print(new_df)

            username                location
0           jplevyak  San Francisco, CA, USA
1           jerblack       New York, NY, USA
2     mm-fc-00000033  San Francisco, CA, USA
3             hirsch        Houston, TX, USA
4              scorz         Denver, CO, USA
5         chadfowler         Denver, CO, USA
6            geoveza         Denver, NY, USA
7              naman  San Francisco, CA, USA
8                jey         Austin, TX, USA
9                 by    Los Angeles, CA, USA
10           snuglas      San Diego, CA, USA
11             bayka        Seattle, WA, USA
12             vladn          Miami, FL, USA
13     thelafffinman      San Diego, CA, USA
14         ericnagel        Buffalo, NY, USA
15          das-mchl       New York, NY, USA
16          newtroll   Montana City, MT, USA
17      constantinos        Bozeman, MT, USA
18           hillale       Rockford, IL, USA
19       kelvintiger         Denver, CO, USA
20              taek         Denver, CO, USA
21        

In [22]:
casts_with_no_parent = session.query(
    Cast).filter(Cast.parent_hash == None).all()

all_associations = session.query(parent_association).all()

all_casts = session.query(Cast).all()

print(len(all_casts) - len(casts_with_no_parent) == len(all_associations))
print(len(all_casts), len(casts_with_no_parent), len(all_associations))


True
100 35 65


In [27]:
# get all users who are in the UK

uk_locations = session.query(Location).filter(
    Location.description.contains("")).all()

uk_users = session.query(User).filter(
    User.location_place_id.in_([location.place_id for location in uk_locations])).all()

# just print the usernames
print(len(uk_users))
for user in uk_users:
    print(user.username)

319
v
dwr
mp
b-rad
sds
julia
shilpa
ertan
mason
brenner
tayyab
osama
liam
jackson
giu
neuroswish
phil
scharf
gary
rz
nickcherry
hunterbm
tikhon
jmj
whatrocks
goose
ted
inkymaze
pzakin
yekim
aman
theodormarcu
markus
hyper
uno
evaav
etekis
dmlb
bgrill
les
vm
timshel
shotbydanni
greg
steph
pushix
jayme
brianjckim
j4ck
pfista
dfern
blake
zachterrell
davisshaver
ccarella
matthew
pdr
mmurthy
bind
frolic
cwe
heershingen
musnit
0xen
alexpaden
briang
emmabwaldron
nonlinear
peter
lara
betashop
dylsteck
cameron
ale
joey
popp
0xshash
olias
mccann
hlau
shef-curry
ninjacongafas
madhur
sahil
salvino
ishika
jpren
drw
nicole
adc266
reyes
adamaziz
scottrepreneur
gabrielayuso
luciano
slobo
nicoglennon
nolan
alecurtu
alvinh
m1guelpf
conabot
mboyle
emmy
nik
ba
hirsh
minh
jake
zachwill
macbudkowski
paulcramer
vinayvasanji
jh
ivy
taylor
dak
0xanu
kamil
cerberus
william
shawn
zrs
july
sophie
cassie
bli
bias
borodutch
amanda
zenboy
jpetrich
encryptedmonk
karmawav
tagga
upside
javi
pl
tylerbenmark
davidtphung
g

In [29]:
# location description of table location is f"{city}, {state}, {country}"
# groupby country, then count

all_location = session.query(Location).all()
unique_countries = set([location.description.split(", ")[-1] for location in all_location])

# for each country, get the number of users
country_user_count = {}
for country in unique_countries:
    country_user_count[country] = len(session.query(User).filter(
        User.location_place_id.in_([location.place_id for location in all_location if location.description.split(", ")[-1] == country])).all())

print(country_user_count)

{'Netherlands': 4, 'Singapore': 3, 'Finland': 1, 'Greece': 1, 'Hong Kong': 1, 'Pakistan': 2, 'Czechia': 1, 'Australia': 5, 'USA': 319, 'Japan': 1, 'Brazil': 2, 'India': 10, 'Mexico': 1, 'Türkiye': 1, 'Nepal': 1, 'Argentina': 1, 'Poland': 1, 'Belgium': 1, 'Portugal': 3, 'Indonesia': 4, 'UK': 8, 'Austria': 1, 'France': 2, 'Spain': 2, 'Canada': 17, 'Italy': 4, 'New Zealand': 2, 'Dubai - United Arab Emirates': 1, 'Sweden': 1, 'Georgia': 1, 'Germany': 2, 'China': 1}


In [46]:
# get all casts with parent, then get parent_association, then diff the length

# get all casts with parent
casts_with_parent = session.query(
    Cast).filter(Cast.parent_hash != None).all()

# get all item in parent_association table
association = session.query(parent_association).all()

print(len(casts_with_parent), len(association))
print(len(casts_with_parent) == len(association))

369326 369326
True
